# Проект определения токсичности комментариев для интеренет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Содержание
1.  Подготовка   
    1.1.  Загрузка данных   
    1.2. Очистка от посторонних символов   
    1.3.  Лемматизация   
    1.4.  Векторизация текстов   
2.  Обучение   
    2.1.  Линейные модели   
    2.2.  Случайный лес   
    2.3.  LightGBM бустинг   
3.  Тестирование лучшей модели   
4.  Выводы   
5.  Чек-лист проверки   

In [1]:
import pandas as pd
from pymystem3 import Mystem
import re
import nltk
import swifter

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore", category=Warning)

## Подготовка

### Загрузка данных

In [2]:
data = pd.read_csv('toxic_comments.csv', index_col=0)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Очистка от посторонних символов

In [3]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text).split()).lower()

In [4]:
data['text'] = data['text'].apply(clear_text)

In [5]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


### Лемматизация

In [6]:
nltk.download('omw-1.4', quiet=True)

True

In [7]:
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)


True

In [8]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


In [9]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(x, get_wordnet_pos(x)) for x in text.split()])


In [10]:
%time data['text'] = data['text'].swifter.apply(lemmatize)


Pandas Apply:   0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: total: 1h 1min 34s
Wall time: 1h 1min 35s


In [11]:
data.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


### Векторизация текстов

* Загрузка словаря стоп-слов
* Выделение таргетного признака
* Разделение на обучающую и тестовую выборки
* Взвешивание терминов tf-idf

In [12]:
nltk.download('stopwords', quiet=True)
stopwords = list(nltk_stopwords.words('english'))

In [13]:
corpus = data['text']
target = data['toxic']

In [14]:
corpus_train, corpus_test, target_train, target_test = train_test_split(corpus, target, test_size=0.2, random_state=13)

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df=50)

## Обучение

### Линейные модели

Логистическая регрессия

In [16]:
log_hp = {'clf__C': [5, 7, 10, 12, 15],
          'clf__max_iter': [500, 750, 1000]}

In [17]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords, min_df=50)),
        ("clf", LogisticRegression()),
    ]
)

In [18]:
logreg_search = GridSearchCV(
    estimator=pipeline,
    param_grid=log_hp,
    n_jobs=3,
    scoring='f1'
)

In [19]:
logreg_search.fit(corpus_train, target_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(min_df=50,
                                                        stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('clf', LogisticRegression())]),
             n_jobs=3,
             param_grid={'clf__C': [5, 7, 10, 12, 15],
                         'clf__max_iter': [500, 750, 1000]},
             scoring='f1')

In [20]:
logreg_search.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(min_df=50,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf', LogisticRegression(C=10, max_iter=500))])

In [21]:
logreg_search.best_score_

0.7626135019341035

Ридж-классификация 

In [22]:
ridge_hp = {
    'clf__alpha': [5, 7, 10, 12, 15],
    'clf__class_weight': ['balanced']
}

In [23]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords, min_df=50)),
        ("clf", RidgeClassifier()),
    ]
)

In [24]:
ridge_search = GridSearchCV(
    estimator=pipeline,
    param_grid=ridge_hp,
    n_jobs=3,
    scoring='f1'
)

In [25]:
ridge_search.fit(corpus_train, target_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(min_df=50,
                                                        stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('clf', RidgeClassifier())]),
             n_jobs=3,
             param_grid={'clf__alpha': [5, 7, 10, 12, 15],
                         'clf__class_weight': ['balanced']},
             scoring='f1')

In [26]:
ridge_search.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(min_df=50,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf', RidgeClassifier(alpha=15, class_weight='balanced'))])

In [27]:
ridge_search.best_score_

0.670647452907835

### Случайный лес


In [28]:
hp = {'clf__max_depth': [2, 4, 6, 8, 10],
      'clf__n_estimators': [200, 400, 600, 1000],
      'clf__class_weight': ['balanced']}

In [29]:
forest_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords, min_df=50)),
        ("clf", RandomForestClassifier()),
    ]
)

In [30]:
fsearch = GridSearchCV(estimator=forest_pipeline,
                     param_grid=hp,
                     scoring='f1',
                     n_jobs=3)

In [31]:
fsearch.fit(corpus_train, target_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(min_df=50,
                                                        stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=3,
             param_grid={'clf__class_weight': ['balanced'],
                         'clf__max_depth': [2, 4, 6, 8, 10],
                         'clf__n_estimators': [200, 400, 600, 1000]},
             scoring='f1')

In [32]:
fsearch.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(min_df=50,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        n_estimators=1000))])

In [33]:
fsearch.best_score_

0.4318189784511476

### LightGBM бустинг

In [47]:
hp1 = {'clf__max_depth': [6, 8, 10, 12], 
       'clf__num_iterations': [200, 500, 1000],
       'clf__num_leaves': [10, 20, 31, 50],
       'clf__class_weight': ['balanced'],
       'clf__verbose': [0]}

In [48]:
lgbm_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords, min_df=50)),
        ("clf", LGBMClassifier()),
    ]
)

In [49]:
lsearch = GridSearchCV(estimator=lgbm_pipeline,
                      param_grid=hp1,
                      scoring='f1',
                      n_jobs=3,
                      )

In [50]:
lsearch.fit(corpus_train, target_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.057429 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(min_df=50,
                                                        stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('clf', LGBMClassifier())]),
             n_jobs=3,
             param_grid={'clf__class_weight': ['balanced'],
                         'clf__max_depth': [6, 8, 10, 12],
                         'clf__num_iterations': [200, 500, 1000],
                         'clf__num_leaves': [10, 20, 31, 50],
                         'clf__verbose': [0]},
             scoring='f1')

In [38]:
lsearch.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(min_df=50,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf',
                 LGBMClassifier(class_weight='balanced', max_depth=12,
                                num_iterations=1000, num_leaves=50))])

In [39]:
lsearch.best_score_

0.7563447413612728

## Тестирование лучшей модели

In [40]:
pred = logreg_search.best_estimator_.predict(corpus_test)

In [41]:
f1_score(target_test, pred)

0.777067921990585

## Выводы

1. Проведена подготовка корпуса текстов:
 * очистка от лишних символов
 * лемматизация
 * tf-idf векторизация
 
2. Результаты метрики f1 для различных моделей, кросс-валидация с подбором гиперпараметров:
    1. Логистическая регрессия: 0,763
    2. Ридж-классификация: 0,671
    3. Случайный лес: 0,431
    4. LightGBM бустинг: 0.756
    
Лучшее значение метрики f1 у модели LogisticRegression(C=12), на тестовой выборке 0,777    